In [69]:
import pandas as pd
import psycopg2
import yaml
from sqlalchemy import create_engine
from sklearn.externals import joblib
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from unidecode import unidecode
from sklearn.externals import joblib
from sklearn.metrics.pairwise import cosine_similarity

import lib.wiki_module as wk
import lib.database_module as db
import lib.encoding_module as em


## First Pipeline

In [3]:
categories = get_args()
categories = ["Automotive technologies"]

## Populate database with categories and pages data from Wikipedia

In [6]:
for cat in categories:
    cat_dict = wk.query_category(cat)
    db.create_or_update_category_in_database(cat_dict["category_id"], cat_dict["category_name"], "local")
    for page in cat_dict["pages"]:
        page_dict = wk.query_page(str(page["page_id"]))
        db.create_or_update_page_in_database(page_dict["page_id"], cat_dict["category_id"], page_dict["title"], 
                                             page_dict["page"],page_dict["summary"],page_dict["html"],"local")


Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connecte

## Encode pages' text

In [5]:
vectorizer = joblib.load("data/vectorizer.pkl")

In [54]:
pages = db.execute_sql_query("select page_id, category_id, page from page", "local")
for page in pages:
    vec = em.get_searchterm_vector(vectorizer, page[2])
    vec = vec.values()[0][0]
    db.execute_sql_statement("""INSERT INTO page_vec (page_id, category_id, page_vec)
                                VALUES ({}, {}, {}) on conflict (page_id) do 
                                UPDATE set category_id = {}, page_vec = {};""".format(page[0],
                                page[1], vec, page[1], vec), "local")


OK
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Connected to server localhost.
Conne

## Second Pipeline

In [83]:
transformer = joblib.load('./data/vectorizer.pkl')

In [84]:
search_term = """A sandwich is a food item consisting of one or more types of food, such as vegetables, 
                 sliced cheese or meat, placed on or between slices of bread, or more generally any dish where 
                 in two or more pieces of bread serve as a container or wrapper for some other food. The sandwich
                 was originally a portable food item or finger food which began to be popular in the Western World.
                 Today sandwiches in various versions are found worldwide.Sandwiches are a popular type of lunch 
                 food, taken to work, school, or picnics to be eaten as part of a packed lunch. The bread can be 
                 used plain, or it can be coated with one or more condiments such as mayonnaise or mustard to 
                 enhance the flavours and texture. As well as being homemade, sandwiches are also widely sold 
                 in restaurants and cafes, and are sometimes served hot as well as cold. There are both savoury 
                 sandwiches, such as deli meat sandwiches, and sweet sandwiches, such as a peanut butter and 
                 jelly sandwich.The sandwich is considered to be the namesake of John Montagu, 4th Earl of Sandwich,
                 because of the claim that he was the eponymous inventor of this food combination. 
                 The Wall Street Journal has described it as Britains "biggest contribution to gastronomy"."""

In [85]:
search_vect = em.get_searchterm_vector(transformer,search_term)
sv = search_vect.values()[0][0]

page_vects = db.select_all_page_vectors()
page_vect = [pv[1] for pv in page_vects]
pv_df = pd.DataFrame(page_vect)



Connected to server joshuacook.me.


## Finding 5 most relative articles using Cosine Similarity

In [86]:
cosine_s = [cosine_similarity(sv.reshape(1, -1), pv)[0][0] for pv in page_vect]

/Users/mazdak/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/mazdak/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/mazdak/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single s

In [91]:
pages_df = pd.DataFrame(page_vects)

In [93]:
pages_df["cosine_s"] = cosine_s

In [100]:
articles_ids = list(pages_df.sort_values(by="cosine_s", ascending=False).head()[0])

[9089348, 82425, 446012, 3884262, 550376]

In [101]:
selected_pages = []
for p_id in articles_ids:
    selected_pages.append(wk.query_page(str(p_id)))
pd.DataFrame(selected_pages)

,html,page,page_id,summary,title
0,"<p>In North America, a <b>breakfast sandwich</...","In North America, a breakfast sandwich is any ...",9089348,"In North America, a breakfast sandwich is any ...",Breakfast sandwich
1,<p>A <b>sandwich</b> is a food item consisting...,A sandwich is a food item consisting of one or...,82425,A sandwich is a food item consisting of one or...,Sandwich
2,"<p>An <b>open sandwich</b>, also known as an <...","An open sandwich, also known as an open face f...",446012,"An open sandwich, also known as an open face f...",Open sandwich
3,"<p>The German word <b>""Butterbrot""</b> (litera...","The German word ""Butterbrot"" (literally: butte...",3884262,"The German word ""Butterbrot"" (literally: butte...",Butterbrot
4,<p>The <b>horseshoe</b> is an open-faced sandw...,The horseshoe is an open-faced sandwich origin...,550376,The horseshoe is an open-faced sandwich origin...,Horseshoe sandwich


## Finding 5 most relative articles using KNearest method

### Results are not as good as the cosine similarity method

In [70]:
ins_row = pv_df.shape[0]
pv_df.loc[ins_row] = svknn = NearestNeighbors(n_neighbors=6)

knn.fit(pv_df)
neighbors_indx = knn.kneighbors(sv.reshape(1, -1))[1][0][1:]

In [71]:
articles_ids = []
for i in neighbors_indx:
    articles_ids.append(page_vects[i][0])
    
selected_pages = []
for p_id in articles_ids:
    selected_pages.append(wk.query_page(str(p_id)))
pd.DataFrame(selected_pages)

,html,page,page_id,summary,title
0,<p><b>Lithuanian Player of the Year</b> is a f...,Lithuanian Player of the Year is a football aw...,8181732,Lithuanian Player of the Year is a football aw...,Lithuanian Footballer of the Year
1,"<p><b>Simit</b> (Turkish: <span lang=""tr"" xml:...","Simit (Turkish: simit, Arabic: سميط‎, translit...",2904185,"Simit (Turkish: simit, Arabic: سميط‎, translit...",Simit
2,<p>The exercise <b>dưỡng sinh</b> or <b>Dưỡng ...,The exercise dưỡng sinh or Dưỡng Sinh (compare...,38485157,The exercise dưỡng sinh or Dưỡng Sinh (compare...,Dưỡng sinh
3,"<p><b>Yogurt</b>, <b>yoghurt</b>, or <b>yoghou...","Yogurt, yoghurt, or yoghourt ( ˈjoʊɡərt or ˈ...",156754,"Yogurt, yoghurt, or yoghourt ( ˈjoʊɡərt or ˈ...",Yogurt
4,<p>Karate terms come almost entirely from Japa...,Karate terms come almost entirely from Japanes...,18007329,Karate terms come almost entirely from Japanes...,List of karate terms


## Third Pipeline

In [20]:
page_vects = db.select_all_page_vectors()

cat_vects = db.select_all_category_vectors()

page_vects_cat = db.execute_sql_query("""select p.page_id, p.page_vec, c.category_id, c.category_name 
                                         from page_vec as p, page_cate as pc, category as c where 
                                         p.page_id = pc.page_id and pc.category_id = c.category_id""")
page_vects_cat_df = pd.DataFrame(page_vects_cat, columns=["page_id", "page_vec", "category_id", "category_name"])

page_vects_cat_df.drop_duplicates("page_id", inplace=True)
page_vects_cat_df["mapped_cat_id"] = pd.factorize(page_vects_cat_df["category_id"])[0]
cats_map = page_vects_cat_df[["mapped_cat_id", "category_name", "category_id"]].drop_duplicates()
cats_map.reset_index(drop=True, inplace=True)

Connected to server joshuacook.me.
Connected to server joshuacook.me.
OK
Connected to server joshuacook.me.


In [28]:
X = list(page_vects_cat_df["page_vec"])
X_df = pd.DataFrame(X)
y = list(page_vects_cat_df["mapped_cat_id"])
X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=22)

In [29]:
classifier = KNeighborsClassifier(n_neighbors=7, n_jobs=-1)
classifier.fit(X_train, y_train)
classifier.score(X_test, y_test)

0.88732394366197187

In [30]:
joblib.dump((classifier, cats_map), 'data/classifier.pkl')

['data/classifier.pkl']

## Forth Pipeline

In [32]:
vectorizer = joblib.load('data/vectorizer.pkl')
transformer, cat_map = joblib.load('data/classifier.pkl')

In [33]:
search_article = "82425"

In [35]:
page_dict = wk.query_page(search_article)
sp_vec = vectorizer.transform([unidecode(page_dict["page"])])[0]
cat_pred = transformer.predict_proba(sp_vec.reshape(1, -1))[0]
cat_map["prob"] = cat_pred

In [42]:
cat_map.sort_values(by="prob", ascending=False)[["category_id", "category_name"]].head(1)

,category_id,category_name
1,757471,Sandwiches


In [39]:
cat_map.sort_values(by="prob", ascending=False)

,mapped_cat_id,category_name,category_id,prob
1,1,Sandwiches,757471,0.857143
13,13,Physical exercise,34955640,0.142857
0,0,influenza,2932559,0.000000
16,16,Physical_quantities,18726608,0.000000
15,15,health care,35880954,0.000000
14,14,desserts,691015,0.000000
12,12,Belief,956054,0.000000
11,11,shoes,2835113,0.000000
10,10,sports terminology,1489690,0.000000
9,9,sports cars,695196,0.000000
